# Power Spectrum with Deterministic and Stochastic Signals 

In [143]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Layout
import ipywidgets as widgets
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:50% !important; }</style>"))

In the following we consider a signal consisting of a deterministic and random part 
$$
x(t)= d(t) + s(t), 
$$
and consider how to derive the power spectrum of $x(t)$. In this particluar example, we consider as the deterministic part a sum of two harmonic signals
$$
d(t) = A_0\sin(\omega_0 t) + A_1\sin(\omega_1 t), 
$$
with known frequencies and zero relative phase. The stochastic signal is white noise with mean zero and unit variance. This is not crucial for the result, but simplifies the calculations. In this case, the normal Fourier transform becomes ill-defined as $s(t)$ is non-integrable in the normal sense. However, we can still calculate the power spectrum by considering the auto-correlation function,$r_{xx}(\tau)$ defined as 
$$
r_{xx}(\tau) = \lim_{T\to\infty}\frac{1}{T} \int_{T/2}^{T/2} x(t)x(t+\tau)dt,
$$
where we have assumed that $x(t)$ is ergodic, which holds for this particular example. This allows us to substitute the sample means with the time integral and thus define the auto correlation function in the same way for both $d(t)$ and $s(t)$. Substituting the expression for $x(t)$ in the integral above, we arrive at 
$$
r_{xx}(\tau) = \lim_{T\to\infty} \frac{1}{T} \left(  \int_{T/2}^{T/2} d(t)d(t+\tau) dt +  \int_{T/2}^{T/2} s(t)s(t+\tau) dt\right),
$$
where we have used that $s(t)$ has zero mean. The last term is the definition of $r_{ss}(\tau)$. The deterministic integral is given by 
$$
\lim_{T\to\infty} \frac{1}{T} \left( \int_{T/2}^{T/2} d(t)d(t+\tau) dt \right) = \frac{A_0^2}{2}\cos(\omega_0\tau) + \frac{A_1^2}{2}\cos(\omega_1\tau), 
$$
where we have used the identity 
$$
\sin(v+w)=\sin v\cos w+\cos v \sin w,
$$
and the fact that 
$$
\lim_{T\to\infty} \frac{1}{T} * \text{oscillating terms} = 0. 
$$

Combining the deterministic and stochastic part, the auto-correlation function is given by 
$$
r_{xx}(\tau) = \frac{A_0^2}{2}\cos(\omega_0\tau) + \frac{A_1^2}{2}\cos(\omega_1\tau) + r_{ss}(\tau). 
$$
We can now take the definition for the power spectrum as the Fourier transform of the auto-correlation function, which is a well defined quantity for a wide sense stationary (WSS) process (again holding true in this particlular example). We thus arrive at the rather intuitively clear result that the power spectrum of $x(t)$ is given by 
$$
P_x(\omega) = \frac{A_0^2}{2}\delta(\omega - \omega_0) + \frac{A_1^2}{2}\delta(\omega - \omega_1) + P_s(\omega), 
$$
which consists of a linear combination of delta peaks at the harmonic frequencies and the power spectrum of the stochastic signal. 


We can verify and experiment with this result by calculating the FFT spectrum of $d(t)$, $x(t)$, and $r_{xx}(\tau)$. Below we plot these signals in the left column. In the right column we plot the corresponding Fourier transforms of the individual signals. 

In [41]:
def plot_func(f0,f1,a0,a1,anoise):
    fs = 20.0
    T = 10
    t = np.arange(0,T,1/fs)
    x = a0*np.sin(2*np.pi*f0*t) + a1*np.sin(2*np.pi*f1*t) + anoise*np.random.randn(len(t))
    xF = np.fft.fft(x)
    N = len(xF)
    xF = xF[0:int(N/2)]
    fr = np.linspace(0,fs/2,int(N/2))
    fig, ax = plt.subplots(2,1, figsize = (10,5))
    fig.suptitle('Signal in time and frequency space')
    ax[0].plot(t,x)
    ax[1].plot(fr,abs(2*xF/(len(t))))
    ax[1].grid()
    ax[0].grid()
    plt.show()

def get_spectrum(sig,fs):
    xF = np.fft.fft(sig)
    N = len(xF)
    xF = xF[0:int(N/2)]
    fr = np.linspace(0,fs/2,int(N/2))
    return 2*xF/len(sig),fr

In [140]:
def plot_func(f0,f1,a0,a1,anoise):

    fs = 20
    T = 10
    t = np.arange(0,T,1/fs)
    #a0 = 1
    #a1 = 1
    #f0 = 1
    #f1 = 2
    #anoise = 3
    sig = a0*np.cos(2*np.pi*f0*t) + a1*np.cos(2*np.pi*f1*t) 
    #sig = a0*np.ones_like(t)

    sig_noise = sig + anoise*np.random.randn(len(t))
    corr = signal.correlate(sig_noise, sig_noise, mode='same', method='direct')
    corr /= np.max(corr)
    sig_freq, fr_sig = get_spectrum(sig,fs)
    sig_noise_freq, fr_sig = get_spectrum(sig_noise,fs)
    power_spec, fr_corr = get_spectrum(corr,fs)
    
    fig, ax = plt.subplots(3, 2, figsize=(10, 10/3*2))
    ax[0,0].plot(t,sig)
    ax[0,0].set_title('Original signal')
    ax[0,0].set_xlabel('Time')
    ax[1,0].plot(t,sig_noise)
    ax[1,0].set_title('Signal with noise')
    ax[1,0].set_xlabel('Time')
    ax[2,0].plot(corr)
    ax[2,0].set_title('Auto-Correlation')
    ax[2,0].set_xlabel('Lag')
    ax[0,0].margins(0, 0.1)
    ax[1,0].margins(0, 0.1)
    ax[2,0].margins(0, 0.1)

    ax[0,1].plot(fr_sig,np.abs(sig_freq))
    ax[0,1].set_title('FFT(Original signal)')
    ax[0,1].set_xlabel('Freq')
    ax[1,1].plot(fr_sig,np.abs(sig_noise_freq))
    ax[1,1].set_title('FFT(Signal with noise)')
    ax[1,1].set_xlabel('Freq')
    ax[2,1].plot(fr_corr,np.abs(power_spec))
    ax[2,1].set_title('FFT(Auto-Correlation)')
    ax[2,1].set_xlabel('Freq')
    ax[0,1].margins(0, 0.1)
    ax[1,1].margins(0, 0.1)
    ax[2,1].margins(0, 0.1)


    fig.tight_layout()
    plt.show()


w = interact(plot_func, f0 = widgets.FloatSlider(value=2,
                                               min=1,
                                               max=5.0,
                                               step=0.5),
                    f1 = widgets.FloatSlider(value=3,
                                               min=1,
                                               max=5.0,
                                               step=0.5),
                    a0 = widgets.FloatSlider(value=1,
                                               min=0,
                                               max=5.0,
                                               step=0.5),
                    a1 = widgets.FloatSlider(value=1,
                                               min=0,
                                               max=5.0,
                                               step=0.5),
                    anoise = widgets.FloatSlider(value=0.1,
                                               min=0,
                                               max=5.0,
                                               step=0.1))


plt.show()

interactive(children=(FloatSlider(value=2.0, description='f0', max=5.0, min=1.0, step=0.5), FloatSlider(value=…

As expected, $r_{xx}(\tau)$ contain the same frequencies as $x(t)$ which can be seen in both the time and frequency domain. The autocorrelation $r_{xx}(\tau)$ is not however a pure sum of two harmonic signals and I am not sure why. Been playing with both integration time and sampling frequency without any change. Food for thought. 